In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

from tqdm.auto import tqdm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [2]:
# one-step sarima forecast
def sarima_forecast(data, config):
    order, sorder, trend = config
    # define model
    model = SARIMAX(
        data,
        order=order,
        seasonal_order=sorder,
        trend=trend,
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    # fit model
    model_fit = model.fit(disp=False)
    return model_fit

In [3]:
# Avarage temp

In [13]:
train = pd.read_csv('../monica/monica_weather_2011-2016.csv', 
                      index_col = ['de-date'], 
                      parse_dates = ['de-date'], 
                      dayfirst = True)
train = train.asfreq('d')

test = pd.read_csv('../monica/reanalysis_2017.csv')

In [14]:
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [15]:
test.head()

,de-date,tavg,tmin,tmax,wind,globrad,precip,relhumid
0,01.01.2017,-3.85,-6.03,-1.72,3.96,2.14,0.90,98.50
1,02.01.2017,-1.89,-3.75,-0.85,5.43,1.92,0.57,96.75
2,03.01.2017,-3.72,-9.02,-1.72,5.07,2.04,0.90,95.38
3,04.01.2017,-6.60,-9.16,-5.06,5.27,3.57,3.88,97.56
4,05.01.2017,-4.36,-6.16,-2.20,3.25,3.15,1.17,97.38


In [16]:
tavg = sarima_forecast(data=train.tavg, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
tmin = sarima_forecast(data=train.tmin, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
tmax = sarima_forecast(data=train.tmax, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
wind = sarima_forecast(data=train.wind, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
globrad = sarima_forecast(data=train.globrad, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
precip = sarima_forecast(data=train.precip, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
relhumid = sarima_forecast(data=train.relhumid, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])

In [17]:
tavg_preds = tavg.predict(start=2*365, end=3*365-1)
tmin_preds = tmin.predict(start=2*365, end=3*365-1)
tmax_preds = tmax.predict(start=2*365, end=3*365-1)
wind_preds = wind.predict(start=2*365, end=3*365-1)
globrad_preds = globrad.predict(start=2*365, end=3*365-1)
precip_preds = precip.predict(start=2*365, end=3*365-1)
relhumid_preds = relhumid.predict(start=2*365, end=3*365-1)

In [18]:
len(tavg_preds)

365

In [19]:
test.tavg = tavg_preds.to_numpy()
test.tmin = tmin_preds.to_numpy()
test.tmax = tmax_preds.to_numpy()
test.wind = wind_preds.to_numpy()
test.globrad = globrad_preds.to_numpy()
test.precip = precip_preds.to_numpy()
test.relhumid = relhumid_preds.to_numpy()

In [20]:
test.to_csv('../monica/sarimax_2017.csv', index=False)